# Day 13 - o1

In [1]:
def solve_machine(aA, aY, bA, bY, Xp, Yp):
    """
    Given the increments for A and B moves (aA, aY) and (bA, bY),
    and the target (Xp, Yp),
    find the minimum cost solution where 0 <= a,b <= 100.
    Returns the minimum cost or None if no solution is found.
    """
    min_cost = None
    for a in range(101):
        for b in range(101):
            # Check if these presses align with the prize
            if a * aA + b * bA == Xp and a * aY + b * bY == Yp:
                cost = 3 * a + b  # cost of pressing A is 3 tokens, B is 1 token
                if min_cost is None or cost < min_cost:
                    min_cost = cost
    return min_cost

def main():
    # Read the input lines
    # We assume input lines come in sets of three lines per machine, exactly in the format described.
    with open("input.txt", "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    # Parse input in groups of three lines:
    # Format:
    # Button A: X+<aA>, Y+<aY>
    # Button B: X+<bA>, Y+<bY>
    # Prize: X=<Xp>, Y=<Yp>
    machines = []
    for i in range(0, len(lines), 3):
        lineA = lines[i]
        lineB = lines[i+1]
        lineP = lines[i+2]

        # Parse lineA
        # e.g. "Button A: X+94, Y+34"
        # We'll split by spaces and commas to extract numbers
        # Format is consistently: Button A: X+<aA>, Y+<aY>
        # We can find '+' and parse the numbers after it.
        partA = lineA.split(":")[1].strip()  # "X+94, Y+34"
        # split by comma
        coordsA = [p.strip() for p in partA.split(",")]
        # coordsA[0] = "X+94", coordsA[1] = "Y+34"
        aA = int(coordsA[0].split("+")[1])
        aY = int(coordsA[1].split("+")[1])

        # Parse lineB: "Button B: X+22, Y+67"
        partB = lineB.split(":")[1].strip()
        coordsB = [p.strip() for p in partB.split(",")]
        bA = int(coordsB[0].split("+")[1])
        bY = int(coordsB[1].split("+")[1])

        # Parse lineP: "Prize: X=8400, Y=5400"
        partP = lineP.split(":")[1].strip()
        coordsP = [p.strip() for p in partP.split(",")]
        Xp = int(coordsP[0].split("=")[1])
        Yp = int(coordsP[1].split("=")[1])

        machines.append((aA, aY, bA, bY, Xp, Yp))

    # Solve each machine
    solutions = []
    for (aA, aY, bA, bY, Xp, Yp) in machines:
        cost = solve_machine(aA, aY, bA, bY, Xp, Yp)
        if cost is not None:
            solutions.append(cost)

    # Determine maximum number of prizes and minimal total cost to get them
    # Since we want to win *all possible* prizes (as many as possible), that just means:
    # The count is the length of solutions, and cost is the sum of them.
    # Each machine is independent, so we just sum all minimal costs.
    max_prizes = len(solutions)
    total_cost = sum(solutions)

    print(max_prizes, total_cost)

if __name__ == "__main__":
    main()


152 29711


## Part 2

In [2]:
def extended_gcd(a, b):
    """Extended Euclidean Algorithm.
    Returns (g, x, y) such that a*x + b*y = g = gcd(a,b).
    """
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return (g, x, y)

def solve_machine(aA, aY, bA, bY, Xp, Yp):
    """
    Solve the system:
    aA*a + bA*b = Xp
    aY*a + bY*b = Yp

    Using linear algebra / Diophantine approach.
    We want nonnegative (a,b) minimizing 3a+b.
    """
    # Form the matrix and vector
    # det of matrix
    det = aA*bY - aY*bA
    if det == 0:
        # Either no solutions or infinite. Check consistency.
        # If ratio of Xp and Yp matches one vector?
        # aA/aY = Xp/Yp and bA/bY must be consistent.
        # This is a rare special case; likely no solution or infinite if consistent.
        # Check if (Xp,Yp) is a scalar multiple of (aA,aY) or (bA,bY)
        # If infinite solutions, we must try to find nonnegative solution that minimize cost.
        # This scenario is uncommon given puzzle constraints. 
        # We'll just return None here for simplicity.
        return None

    # Check if we can solve using Cramer's rule directly:
    # a = (Xp*bY - Yp*bA)/det
    # b = (Yp*aA - Xp*aY)/det
    # But we must ensure integrality and nonnegativity.
    num_a = Xp*bY - Yp*bA
    num_b = Yp*aA - Xp*aY

    if num_a % det != 0 or num_b % det != 0:
        # No direct integral solution from Cramer's rule.
        # We'll use a more general Diophantine approach.

        # Solve the vector equation:
        # [aA bA] [a] = [Xp]
        # [aY bY] [b]   [Yp]

        # Consider the system as:
        # aA*a + bA*b = Xp  ...(1)
        # aY*a + bY*b = Yp  ...(2)

        # We can try expressing one variable in terms of the other and use gcd-based reasoning.
        # Another method: 
        # Treat (aA,aY) and (bA,bY) as basis vectors of a lattice. We want to express (Xp,Yp) = m*(aA,aY)+n*(bA,bY).
        # Solve:
        # aA*m + bA*n = Xp
        # aY*m + bY*n = Yp
        # This is the same system, just renaming a->m, b->n.

        # Let's use matrix inverse approach via extended_gcd on det:
        # Actually, we already know no direct division: we must find a particular solution using Extended Euclid on pairs.

        # A more straightforward approach:
        # We'll try to solve one of the equations for a and substitute into the other, then use extended_gcd.

        # From (1): a = (Xp - bA*b)/aA if aA divides (Xp - bA*b).
        # That's not efficient for large numbers without a strategy.

        # Instead, let's do a generic approach:
        # We have 2 equations and 2 unknowns. We know a solution exists if gcd conditions are met.
        # Combine the equations to eliminate one variable using a known technique:
        # Multiply (1) by aY and (2) by aA:
        # aA*a*aY + bA*b*aY = Xp*aY
        # aY*a*aA + bY*b*aA = Yp*aA
        # Subtract them to eliminate a:
        # bA*b*aY - bY*b*aA = Xp*aY - Yp*aA
        # b*(bA*aY - bY*aA) = Xp*aY - Yp*aA
        # b*( - (aY*bA - aA*bY)) = Xp*aY - Yp*aA
        # b*(aY*bA - aA*bY) = Yp*aA - Xp*aY = num_b (we have this from above)
        # Since det = aA*bY - aY*bA, we have aY*bA - aA*bY = -det
        # So b*(-det) = num_b
        # b = -num_b/det

        # Similarly:
        # a = num_a/det

        # We did this already and found they weren't divisible.
        # If not divisible, then no solution. The puzzle states solutions exist for #2 and #4 after shifting, so let's trust integrality might actually hold.

        # Given the puzzle statement, if we hit this path, let's return None.
        return None
    else:
        # We have an integral solution from Cramer:
        a0 = num_a // det
        b0 = num_b // det

        # a0,b0 is a particular solution. Check if a0,b0 >=0:
        if a0 >=0 and b0>=0:
            # Already nonnegative
            return 3*a0+b0
        # If not nonnegative, we must see if we can "adjust" the solution.
        # However, since det !=0, we have a unique solution for (a,b).
        # If that unique integral solution isn't nonnegative, there's no nonnegative solution.
        return None

def main():
    # Read input
    with open("input.txt", "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    machines = []
    for i in range(0, len(lines), 3):
        lineA = lines[i]
        lineB = lines[i+1]
        lineP = lines[i+2]

        # Parse A line
        # Example: "Button A: X+94, Y+34"
        partA = lineA.split(":")[1].strip()  # "X+94, Y+34"
        coordsA = [p.strip() for p in partA.split(",")]
        aA = int(coordsA[0].split("+")[1])
        aY = int(coordsA[1].split("+")[1])

        # Parse B line
        partB = lineB.split(":")[1].strip()
        coordsB = [p.strip() for p in partB.split(",")]
        bA = int(coordsB[0].split("+")[1])
        bY = int(coordsB[1].split("+")[1])

        # Parse Prize line
        partP = lineP.split(":")[1].strip()
        coordsP = [p.strip() for p in partP.split(",")]
        Xp = int(coordsP[0].split("=")[1])
        Yp = int(coordsP[1].split("=")[1])

        # Add the large offset:
        OFFSET = 10_000_000_000_000
        Xp += OFFSET
        Yp += OFFSET

        machines.append((aA,aY,bA,bY,Xp,Yp))

    solutions = []
    for (aA,aY,bA,bY,Xp,Yp) in machines:
        cost = solve_machine(aA,aY,bA,bY,Xp,Yp)
        if cost is not None:
            solutions.append(cost)

    print(len(solutions), sum(solutions))

if __name__ == "__main__":
    main()


168 94955433618919
